In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[2]:


get_ipython().system('jupyter nbconvert   --to script land_value.ipynb')
get_ipython().system("awk '!/ipython/' land_value.py >  temp.py && mv temp.py app.py && rm land_value.py")
get_ipython().system('streamlit run app.py')


# In[ ]:


'''
# Projet Prédiction de valeurs foncières
'''


# In[ ]:


'''
## Préparation des données
'''


# In[ ]:


import pandas as pd
import numpy as np
import random

# Load the dataset
url = "https://www.data.gouv.fr/fr/datasets/r/78348f03-a11c-4a6b-b8db-2acf4fee81b1"
dtypes = {
    'Date mutation': 'str',
    'Nature mutation': 'str',
    'Valeur fonciere': 'float',
    'Code postal': 'str',
    'Commune': 'str',
    'Code departement': 'str',
    'Code commune': 'str',
    'Type local': 'str',
    'Surface reelle bati': 'float',
    'Nombre pieces principales': 'Int64',
    'Surface terrain': 'float',
    'Nature culture': 'str',
    'Nature culture speciale': 'str'
}

#df = pd.read_csv(url, delimiter='|', decimal=",", na_values=[''], skiprows=lambda i: i>0 and random.random() > 0.2) #version plus rapide pour machine learning
df = pd.read_csv(url, delimiter='|', decimal=",", na_values=[''])
df['Nature culture'] = df['Nature culture'].fillna("NaN") #Remplacer les valeurs nulles par un string pour encodage
df['Nature culture speciale'] = df['Nature culture speciale'].fillna("NaN")
df['Type local'] = df['Type local'].fillna("NaN")
#df['Surface reelle bati'] = df['Surface reelle bati'].fillna(df['Surface reelle bati'].mean())
df['Nombre pieces principales'] = df['Nombre pieces principales'].fillna(df['Nombre pieces principales'].mean().round())
#df['Surface terrain'] = df['Surface terrain'].fillna(df['Surface terrain'].mean())

df = df.astype(dtypes)

#garde uniquement le département 92
#df = df[df['Code departement'] == '92']

# Convertir 'Date mutation' en mois (float)
df['Date mutation'] = pd.to_datetime(df['Date mutation'], format='%d/%m/%Y')
df['Date mutation'] = df['Date mutation'].dt.month

print(df.head())
print(df.dtypes)


# 

# In[ ]:


# Garder les colonnes pertinentes
columns_to_keep = [
    'Date mutation', 'Nature mutation', 'Valeur fonciere', 'Code postal',
    'Code departement', 'Type local',
    'Surface reelle bati', 'Nombre pieces principales', 'Surface terrain',
]
df = df[columns_to_keep]


df = df.dropna(subset=['Valeur fonciere', 'Code postal', 'Surface reelle bati', 'Surface terrain'])
# supression valeurs aberrantes
df = df[df['Valeur fonciere']>10]
df = df[df['Valeur fonciere']<5000000]
df = df[df['Surface reelle bati']<100000]
df = df[df['Surface terrain']<400000]
df = df[df['Nombre pieces principales']<40]

print(df.head())


# In[ ]:


"""## Partie Analyse Exploratoire Des Données"""


# In[ ]:


print(df.dtypes)


# In[ ]:


print(df.head())


# In[ ]:


import seaborn as sns
import matplotlib.pyplot as plt

# Histogramme des valeurs foncières
plt.figure(figsize=(10, 6))
sns.histplot(df['Valeur fonciere'], bins=50, kde=True)
plt.title('Distribution des Valeurs Foncières')
plt.xlabel('Valeur Fonciere')
plt.ylabel('Fréquence')
plt.show()

# Boxplot des valeurs foncières par type de location
plt.figure(figsize=(12, 8))
sns.boxplot(x='Type local', y='Valeur fonciere', data=df)
plt.title('Valeur Foncière par Type de Local')
plt.xticks(rotation=45)
plt.show()

# Scatterplot Valeur foncière vs Surface réelle bâti
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Surface reelle bati', y='Valeur fonciere', data=df)
plt.title('Valeur Foncière vs Surface Réelle Bati')
plt.xlabel('Surface Réelle Bati (m²)')
plt.ylabel('Valeur Fonciere')
plt.ylim(0, 5000000)
plt.xlim(0, 10000)
plt.show()


# On retiendra que:
# 
# -La majorité des ventes ont un montant inférieur à 1M d'euros
# -Les maisons et locaux coûtent en général moins cher que les appartements. Cela s'explique par leur positionnement: les appartements se situent surtout en ville, tandis que la majorité des maisons se situe en campagne et banlieue. Les locaux commerciaux et industriels coûtent aussi plus cher par leur nature et leur taille.

# In[ ]:


"""## Geomap"""


# In[ ]:


import geopandas as gpd
import plotly.express as px


# Feature engineering
df = df[df['Surface reelle bati'] > 0]

df['Prix_m2'] = df['Valeur fonciere'] / df['Surface reelle bati']

# Filtrer les valeurs aberrantes
df = df[df['Prix_m2'] < 30000]

prix_m2_par_departement = df.groupby('Code departement')['Prix_m2'].mean().reset_index()
prix_m2_par_departement.columns = ['code', 'Prix_m2']

# Convertir le code département en string pour correspondre au geojson
prix_m2_par_departement['code'] = prix_m2_par_departement['code'].str.zfill(2)

# Formater les prix avec séparateurs de milliers et le symbole €
prix_m2_par_departement['Prix_m2_format'] = prix_m2_par_departement['Prix_m2'].apply(lambda x: f"{x:,.2f} €".replace(',', ' ').replace('.', ','))

# Geomap
geojson_url = 'https://france-geojson.gregoiredavid.fr/repo/departements.geojson'
departements_geojson = gpd.read_file(geojson_url)

# Créer la carte
fig = px.choropleth_mapbox(
    prix_m2_par_departement,
    geojson=departements_geojson,
    featureidkey="properties.code",
    locations='code',
    color='Prix_m2',
    color_continuous_scale="Viridis",
    mapbox_style="carto-positron",
    zoom=5,
    center={"lat": 46.603354, "lon": 1.888334},
    opacity=0.6,
    labels={'Prix_m2': 'Prix moyen par m²'},
    title='Prix moyen par m² par département',
    hover_data={'Prix_m2_format': True, 'Prix_m2': False}  # Utiliser la colonne formatée pour l'affichage
)

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()


# On remarque que les prix moyens par département les plus élevés se situent à Paris et dans le Sud-Est de la France.

# In[ ]:


"""## Histogramme"""


# In[ ]:


df.groupby('Code departement')['Prix_m2'].mean().plot(kind='bar', figsize=(30, 10))
plt.ylabel('Prix au m² moyen')
plt.title('Prix au m² par département')
plt.show()


# In[ ]:


"""## Encodage des Variables Catégorielles"""


# In[ ]:


from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Encoder les colonnes catégorielles en utilisant LabelEncoder
label_encoders = {}
df_encoded = df.copy()
for col in df.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df_encoded[col] = le.fit_transform(df[col])
    label_encoders[col] = le

#standardiser
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.set_output(transform='pandas')
df_scaled = scaler.fit_transform(df_encoded)

#Matrice de corrélation
correlation_matrix = df_scaled.corr()
plt.figure(figsize=(15, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Matrice de Corrélation')
plt.show()


# In[ ]:


"""## Scatter matrix"""


# In[ ]:


from pandas.plotting import scatter_matrix
scatter_matrix(df_scaled, figsize=[20, 20])
plt.show()


# In[ ]:


columns_to_keep = ['Surface reelle bati', 'Surface terrain', 'Nombre pieces principales', 'Type local', 'Nature mutation']

fig, axes = plt.subplots(nrows=len(columns_to_keep) - 1, ncols=1, figsize=(10, len(columns_to_keep) * 4))
fig.subplots_adjust(hspace=0.5)

for i, col in enumerate(columns_to_keep):
    if col != 'Valeur fonciere':
        ax = axes[i - 1]
        ax.scatter(df[col], df['Valeur fonciere'])
        ax.set_ylabel('Valeur fonciere')
        ax.set_xlabel(col)
        ax.set_title(f'Valeur fonciere vs {col}')

plt.show()


# In[ ]:


"""# Partie Modeles Machine Learning"""


# In[ ]:


"""## Apprentissage non supervisé"""


# In[ ]:


"""## Méthode du coude"""


# In[ ]:


from sklearn.cluster import KMeans
# Méthode du coude pour déterminer le nombre optimal de clusters
sse = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(df_scaled)
    sse.append(kmeans.inertia_)

plt.figure(figsize=(10, 6))
plt.plot(range(1, 11), sse, marker='o')
plt.xlabel('Nombre de Clusters')
plt.ylabel('SSE')
plt.title('Méthode du Coude')
plt.show()


# In[ ]:


"""## KMeans"""


# In[ ]:


#clustering
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(df_scaled)
labels = kmeans.labels_

#plot valeur fonciere against sufrace bati
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Valeur fonciere', y='Surface reelle bati', hue=labels, data=df)
plt.ylim(0, 400)
plt.xlim(0, 5000000)
plt.title('Valeur Fonciere vs Surface Réelle Bati')
plt.xlabel('Valeur Fonciere')
plt.ylabel('Surface Réelle Bati')
plt.show()

#boxplot de chaque colonne avec label
plt.figure(figsize=(10, 6))
sns.boxplot(x='Valeur fonciere', data=df, hue=labels)
plt.show()

plt.figure(figsize=(10, 6))
sns.boxplot(x='Surface reelle bati', data=df, hue=labels)
plt.show()

plt.figure(figsize=(10, 6))
sns.boxplot(x='Surface terrain', data=df, hue=labels)
plt.show()

plt.figure(figsize=(10, 6))
sns.boxplot(x='Nombre pieces principales', data=df, hue=labels)
plt.show()

plt.figure(figsize=(10, 6))
sns.boxplot(x='Type local', data=df, hue=labels)
plt.show()

plt.figure(figsize=(10, 6))
sns.boxplot(x='Code postal', data=df, hue=labels)
plt.show()

plt.figure(figsize=(10, 6))
sns.boxplot(x='Date mutation', data=df, hue=labels)
plt.show()


# In[ ]:


#print how many points in each cluster
for i in range(4):
    print(f"Cluster {i}: {np.sum(labels == i)} points")


# In[ ]:


"""## Apprentissage supervisé"""


# In[ ]:


"""### KNN"""


# In[ ]:


# Utilisation de KNN en utilisant le label code postal

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Sélection des features et de la cible pour le KNN
X = df_scaled.drop('Code departement', axis=1)
y = df_encoded['Code departement']

# Division des données en ensembles d'apprentissage et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialisation et entrainement du modèle KNN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# Prédictions
y_pred = knn.predict(X_test)

# Évaluation du modèle
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))




# In[ ]:


from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Sélection des features et de la cible pour le KNN
X = df_scaled.drop('Type local', axis=1)
y = df_encoded['Type local'].astype(int)  # Conversion en entier si nécessaire

# Division des données en ensembles d'apprentissage et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialisation et entrainement du modèle KNN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# Prédictions
y_pred = knn.predict(X_test)

# Évaluation du modèle
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


# In[ ]:


"""### Régression linéaire"""


# In[ ]:


# Utilisation d'une régression linéaire

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Sélection des features et de la cible pour la régression linéaire
X = df_scaled.drop('Valeur fonciere', axis=1)
y = df_encoded['Valeur fonciere']

# Division des données en ensembles d'apprentissage et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialisation et entrainement du modèle de régression linéaire
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

print("Random Forest Mean Squared Error:", mean_squared_error(y_test, y_pred_rf))
print("Random Forest R^2 Score:", r2_score(y_test, y_pred_rf))

plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_rf, alpha=0.3)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', lw=2)
plt.xlabel('Valeur Fonciere Réelle')
plt.ylabel('Valeur Fonciere Prédite')
plt.title('Random Forest : Valeur Fonciere Réelle vs Prédite')
plt.show()

